# Getting the Data

- To retrieve the tweets in the twitter we have to first open an account in the twitter and then login with the same account in the `developers.twitter.com`.

- After that create a new app and get the consumer API key and secret also the Access token and secret.


In [2]:
# ! pip install tweepy

In [3]:
import pandas as pd 
import tweepy
import json
from tweepy import OAuthHandler

In [4]:
# We got this consumer and access, keys and tokens in the developer.twitter.com by creating a new app in that.

consumer_key = "KfrreskF0adWeYag3whgQRtq5"
consumer_secret = "CbtwJhmZX7W7T4R1j0g9BVeDXdwMhINfu9tuA33jnHpaKdl9cU"

access_token = "1069493766493396992-m53lv11lu6D7Ow5CbUzkbBHjwe3Lcz"
access_token_secret = "73EBN0GnkZ7sg1xZNIsnv85zmxTZPIRepfITKL17jp8vF"

In [6]:
# Creating OAuthHandler object with the consumer key and secret
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)

#Creating the access by setting the access token and secret  
auth.set_access_token(access_token,access_token_secret) 

#Creating and api object for the auth handler
api = tweepy.API(auth_handler=auth)

# Sample retrieval of tweets

In [13]:
query = "ipl 2020"

In [14]:
tweets = api.search(query,count=10,lang='en',exclude='retweets',tweet_mode='extended')

# Printing the tweets one by one
for tweet in tweets:
    print(tweet,'\n')
    print('........................................\n')

Status(_api=<tweepy.api.API object at 0x05ED1790>, _json={'created_at': 'Thu Nov 05 07:41:46 +0000 2020', 'id': 1324255570153463808, 'id_str': '1324255570153463808', 'full_text': 'IPL #Eliminator: #Momentum with #SRH, but #RCB has the fire power https://t.co/P4WaISIt5P', 'truncated': False, 'display_text_range': [0, 89], 'entities': {'hashtags': [{'text': 'Eliminator', 'indices': [4, 15]}, {'text': 'Momentum', 'indices': [17, 26]}, {'text': 'SRH', 'indices': [32, 36]}, {'text': 'RCB', 'indices': [42, 46]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/P4WaISIt5P', 'expanded_url': 'https://www.10net.in/2020/11/05/ipl-eliminator-momentum-with-srh-but-rcb-has-the-fire-power/', 'display_url': '10net.in/2020/11/05/ipl…', 'indices': [66, 89]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_t

**NOTE**:
- In the above cell we retrieved the 10 tweets regarding the `query` of "ipl 2020"
- We can see that the tweets retrieved are in the `json` format
- It contains numerous informations in it like the friends count , followers count , user informations , date and time of the tweet etc..,


## Getting the tweets and storing in the dataframe

- Getting the tweets and some other attributes from the retrieved json format and storng it in the excel file

In [15]:
df = pd.DataFrame(columns= ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date


The above function will retrieve maximum of 1000 tweets regarding the query which is mentioned.

In [56]:
def stream(data,filename):
    '''
    This function will take the following parameters
    data = which is the query to search for  
    filename = the name of the excel to be stored
    
    Attributes - data , filename
    '''
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en',exclude = 'retweets').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        
        i = i+1
        
        if i == 1000:
            df.to_excel(f'{filename}.xlsx')
            break
        else:
            pass

In [57]:
stream(data=['USA election'],filename='usa_election_tweets')

In [58]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,"Uber, gig economy companies see California win...",Marketplace Risk,3084,1191,"San Francisco, CA",False,0,0,2020-11-05 09:40:11
1,Why is twitter still suppressing material abou...,100% ( livin in Covid19 hell ),90981,3635,"Wales, England",False,0,0,2020-11-05 09:40:01
2,Epidemic proportion. Trash this supposed US Pr...,Deborah Ann Morel,3511,616,,False,0,0,2020-11-05 09:39:55
3,Hey guys. What is your opinion about Election ...,Przemyslaw Szymczak,196,42,"Nottingham, England",False,0,0,2020-11-05 09:39:24
4,"Been watching USA elections from far away, ver...",🎲,335,12,,False,0,0,2020-11-05 09:38:56


## Analysing the sentiment using the TextBlob

Sentiment(polarity, subjectivity)

Polarity score: [-1 , +1]
    
    -1 = negative tweet
     0 = means neutral tweet 
    +1 = means positive tweet

Subjectivity: [0.0 , 1.0] 

where 0.0 is very Objective and 1.0 is very Subjective.

In [54]:
# !pip install textblob

In [60]:
from textblob import TextBlob

In [59]:
import re

def clean_tweet(tweet):
    '''
    Removes all the mentions(@),Hashtags(#),URL and also 
    anything other than the letters and number will be 
    removed and replaced with space 
    '''
    
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [73]:
def analyze_sentiment(tweet):
    '''
    Analyzes the sentiment of the tweet and returns whether the tweet is 
    positive , negative or neutral.
    '''
    analysis = TextBlob(tweet)
    
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'
        
    
    

In [66]:
df['clean_tweet'] = df.Tweets.apply(lambda x: clean_tweet(x))

In [67]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,"Uber, gig economy companies see California win...",Marketplace Risk,3084,1191,"San Francisco, CA",False,0,0,2020-11-05 09:40:11,Uber gig economy companies see California win ...
1,Why is twitter still suppressing material abou...,100% ( livin in Covid19 hell ),90981,3635,"Wales, England",False,0,0,2020-11-05 09:40:01,Why is twitter still suppressing material abou...
2,Epidemic proportion. Trash this supposed US Pr...,Deborah Ann Morel,3511,616,,False,0,0,2020-11-05 09:39:55,Epidemic proportion Trash this supposed US Pre...
3,Hey guys. What is your opinion about Election ...,Przemyslaw Szymczak,196,42,"Nottingham, England",False,0,0,2020-11-05 09:39:24,Hey guys What is your opinion about Election i...
4,"Been watching USA elections from far away, ver...",🎲,335,12,,False,0,0,2020-11-05 09:38:56,Been watching USA elections from far away very...


In [68]:
df['Tweets'][0]

'Uber, gig economy companies see California win as national model https://t.co/YguGYZc8do #gigeconomy #technews #Prop22 #apps (via @Reuters )'

In [69]:
df['clean_tweet'][0]

'Uber gig economy companies see California win as national model gigeconomy technews Prop22 apps via'

In [74]:
df['sentiment'] = df['clean_tweet'].apply(lambda x: analyze_sentiment(x))

In [75]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,sentiment
0,"Uber, gig economy companies see California win...",Marketplace Risk,3084,1191,"San Francisco, CA",False,0,0,2020-11-05 09:40:11,Uber gig economy companies see California win ...,Positive
1,Why is twitter still suppressing material abou...,100% ( livin in Covid19 hell ),90981,3635,"Wales, England",False,0,0,2020-11-05 09:40:01,Why is twitter still suppressing material abou...,Negative
2,Epidemic proportion. Trash this supposed US Pr...,Deborah Ann Morel,3511,616,,False,0,0,2020-11-05 09:39:55,Epidemic proportion Trash this supposed US Pre...,Positive
3,Hey guys. What is your opinion about Election ...,Przemyslaw Szymczak,196,42,"Nottingham, England",False,0,0,2020-11-05 09:39:24,Hey guys What is your opinion about Election i...,Positive
4,"Been watching USA elections from far away, ver...",🎲,335,12,,False,0,0,2020-11-05 09:38:56,Been watching USA elections from far away very...,Positive


In [85]:
n = 642

print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['sentiment'][n])

Original tweet:
 @QueenAzizzar @057_CaptainFS The strain on my mans right eye is USA election day level 😂😂😂

Ignore me just saluting a king kinging

Clean tweet:
 CaptainFS The strain on my mans right eye is USA election day level Ignore me just saluting a king kinging

Sentiment of the tweet:
 Positive


In [82]:
df[df['sentiment'] == 'Neutral'].shape[0]

387

In [83]:
df[df['sentiment'] == 'Positive'].shape[0]

433

In [84]:
df[df['sentiment'] == 'Negative'].shape[0]

180